In [4]:
import os
from dotenv import load_dotenv, find_dotenv

load_dotenv(find_dotenv())

from langchain_openai import OpenAIEmbeddings, OpenAI
from langchain_core.prompts import PromptTemplate

model_name = os.getenv("LLM_MODEL") or "gpt-4o-mini"
model_provider = os.getenv("LLM_PROVIDER") or "openai"

import bs4
from langchain import hub
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.vectorstores import Chroma
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_openai import OpenAIEmbeddings, ChatOpenAI



In [ ]:
#### INDEXING ####
loader = WebBaseLoader(
    web_paths=["https://news.naver.com/section/101"],
    bs_kwargs=dict(
        parse_only=bs4.SoupStrainer(
            class_=("sa_text", "sa_item SECTION_HEADLINE")
        )
    )
)

docs = loader.load()

In [6]:
# Split
text_splitter = RecursiveCharacterTextSplitter.from_tiktoken_encoder(
    encoding_name="cl100k_base",
    chunk_size=300,
    chunk_overlap=50
)
splits = text_splitter.split_documents(docs)

In [7]:
vectorstore = Chroma.from_documents(
    documents=splits,
    embedding=OpenAIEmbeddings(
        model="text-embedding-3-large",
        
    )
)

In [24]:
import re
def clean_whitespace(text):
    """연속된 white space를 하나로 만드는 함수"""
    # 연속된 공백을 하나의 공백으로 줄임
    text = re.sub(r'( )+', ' ', text)
    # 연속된 개행을 하나의 개행으로 줄임
    text = re.sub(r'\n+', '\n', text)
    # 문자열 양쪽 끝의 공백을 제거
    text = text.strip()

    return text

In [25]:
#print(docs)

for doc in docs:
    print(clean_whitespace(doc.page_content))
    print('meta:', doc.metadata)
    print("-"*100)


최태원 “성과급 5천% 는다고 행복 아냐”…SK하이닉스 성과급 갈등 언급
최태원 SK그룹 회장이 SK하이닉스에서 격화되고 있는 성과급 갈등과 관련해 “(성과급이) 3천%, 5천%까지 늘어난다고 행복해지는 것은 아니다”라고 말했습니다. 최 회장은 어제(20일) 직원들과의 대화 ‘슬기로운 S
KBS
73
개의 관련뉴스 더보기
전남도,2조7천억‘국가 수소특화단지’도전
영광 대마산단에 국내 최대규모 조성위한 산업부 공모 신청 핵심전략은 무탄소 전력 활용 경제적·안정적 청정수소 생산 전력 계통 포화문제 완화로 재생에너지 활성화 기여 차별화 안내도 [헤럴드경제(전남)=김경민기자]전남도
헤럴드경제
27
개의 관련뉴스 더보기
"보따리부터 내놓으란 석화업계…뼈깎는 자구노력 없인 금융지원 없다"
금융당국이 ‘자기 뼈를 깎는 자구노력’이라는 석유·화학업종 자금지원 대원칙을 천명하고 구조개편에 필요한 자금공급 방안을 준비한다. 기업이 어려워지면 은행권에 손부터 내미는 관행에 칼을 빼드는 한편, 은행권에는 ‘생산
이데일리
65
개의 관련뉴스 더보기
트럼프가 '콕' 찍었다…"9만전자 간다" KB증권 분석 보니
도널드 트럼프 미국 행정부가 삼성전자 지분 확보를 검토하는 것으로 알려지면서, 삼성전자에 대한 미국 정부와의 결속 강화 가능성이 점쳐지며 증권가 목표주가가 '9만원'을 유지했다. 21일 KB증권은 “삼성전자의 하반기
서울경제
22
개의 관련뉴스 더보기
롯데하이마트, 여름특가세일…“최대 80만원 캐시백”
롯데하이마트가 계속되는 무더위에 오는 8월 말까지 여름 마지막 특가 세일을 진행한다. 롯데하이마트는 이달 말까지 여름 특가 세일을 한다고 22일 밝혔다. 에어컨의 경우 캐시백과 증정품 행사 등을 진행한다. LG전자
헤럴드경제
9
개의 관련뉴스 더보기
3대 혁신경영 잇는 정의선 "고객 만족이 핵심, 우린 항상 진화한다"
미래를 만드는 주체는 고객이고, 그들의 목소리에 진정으로 귀 기울이고 응답하는 것이 우리의 역할." 정의선 현대차그룹 회장은 조부 고(故) 정주영 창업회장의 이같은 

In [10]:
retriever = vectorstore.as_retriever(
    search_type="mmr",
    search_kwargs=dict(
        k=1,
        fetch_k=4
    )
)



In [13]:
prompt = hub.pull("sungwoo/ragbasic")
print(prompt)

input_variables=['context', 'question'] input_types={} partial_variables={} metadata={'lc_hub_owner': 'sungwoo', 'lc_hub_repo': 'ragbasic', 'lc_hub_commit_hash': 'a53edda0c0e62edb01c0912f1db21b9083b3638197bd512e3d8c10e308b8dddf'} messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context', 'question'], input_types={}, partial_variables={}, template='질문-답변 작업을 돕는 비서입니다. 질문에 답하기 위해 다음과 같은 검색 결과를 사용하세요. 답을 모르는 경우에는 모른다고 명시하세요. 답변은 세 문장 이내로 간결하게 유지해 주세요.\n질문: {question}\n검색 결과: {context}\n답변:'), additional_kwargs={})]


In [ ]:
import re

llm = ChatOpenAI(model=model_name, temperature=0)

def format_docs(docs):
    formatted = "\n\n".join(doc.page_content for doc in docs)
    return formatted
    

In [15]:
# Chain
rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

print(rag_chain.invoke("오늘 뉴스 중 중요한 것은 무엇인가?"))

오늘 중요한 뉴스는 이재명 대통령과 도널드 트럼프 미국 대통령의 첫 한미 정상회담을 앞두고 조선주와 원전주가 동반 상승했다는 소식입니다. 특히 한화오션 주가가 크게 올랐습니다.


In [26]:
docs = rag_chain.invoke("국채 관련한 정보를 알려줘")
print(docs)

국채는 정부가 재정을 조달하기 위해 발행하는 채권입니다. 최근 기업들이 추석 전 자금 확보를 위해 회사채를 대규모로 발행하고 있습니다. 국채는 안정적인 투자 수단으로 여겨집니다.


In [28]:
from langchain.prompts import ChatPromptTemplate

template = """
당신은 AI 언어 모델 조수입니다. 당신의 임무는 주어진 사용자 질문에 대해 벡터 데이터베이스에서 관련 문서를 검색할 수 있도록 다섯 가지 다른 버전을 생성하는 것이빈다.
사용자 질문에 대한 여러 관점을 생성함으로써, 거리 기반 유사성 검색의 한계를 극복하는 데 도움을 주는 것이 목표입니다.
각 질문은 새로운 줄로 구분하여 제공하세요. 원본 질문: {question}
"""

prompt_perspectives = ChatPromptTemplate.from_template(template)

from langchain_core.output_parsers import StrOutputParser
from langchain_openai import ChatOpenAI

generate_queries = (
    prompt_perspectives
    | ChatOpenAI(model=model_name, temperature=0)
    | StrOutputParser()
    |(lambda x: x.split("\n"))
)

generated_query = generate_queries.invoke("집값의 향방?")
print(generated_query)



['집값은 앞으로 어떻게 변할까요?  ', '향후 부동산 가격 동향은 어떨지 궁금합니다.  ', '주택 시장의 가격 추세는 앞으로 어떤 방향일까요?  ', '집값 상승 또는 하락 전망에 대해 알고 싶습니다.  ', '다가오는 시기에 부동산 가격이 어떻게 움직일지 예측할 수 있을까요?']


In [29]:
from langchain.load import dumps, loads

def get_unique_union(documents: list[list]):
    """
    고유한 문서들의 합집합을 생성하는 함수입니다.
    """

    flattened_docs = [dumps(doc) for sublist in documents for doc in sublist]

    unique_docs = list(set(flattened_docs))

    return [loads(doc) for doc in unique_docs]


In [30]:
question = "집값의 향방?"

In [31]:
retrievel_chain = generate_queries | retriever.map() | get_unique_union

In [32]:
docs = retrievel_chain.invoke(question)

In [35]:
print(len(docs))
print(docs)

2
[Document(metadata={'source': 'https://news.naver.com/section/101'}, page_content='"입찰보증금 1000억원 쉽지않네"…성수1지구, 3파전 관측 속 이변 가능성\n\n사업비 2조원에 달하는 성수1지구의 시공권을 따내기 위한 수주전의 막이 올랐다. 경기 침체 등으로 건설사의 정비사업 선별 수주가 대세인 상황에서, 한강벨트 재개발 사업지를 수주하기 위해 현대건설과 GS건설, HDC현\n\n\n아시아경제\n\n26분전\n\n\n\n\n\n\n\n\n코스피, 잭슨홀 회의 경계감에도 장 초반 1%대 상승세(종합)'), Document(metadata={'source': 'https://news.naver.com/section/101'}, page_content='한국소비자원 주식 투자자 2100명 조사 증권사 MTS 불만은 늘고 만족도 하락 ‘빨리 팔아야 하는데, 먹통이라니…’ 주식거래를 하다가 갑자기 앱이 먹통되면 혼란스러울 수 밖에 없는데, 최근 증권사 주식거래 앱 사용\n\n\n매일경제\n\n21분전\n\n\n\n\n\n\n\n\n30대 맞벌이 부부, 월 131만원으로 미래 설계하기 [재테크 Lab]\n\n상담자 중에선 투자상품으로 얼마만큼의 수익을 내느냐에만 관심 갖는 이들이 적지 않다. 하지만 간과해선 안 되는 게 있다. 바로 절세다. 같은 투자 성과를 내도 세금 공제가 얼마만큼 되느냐에 따라 최종 수익이 확연하게')]


In [36]:
from langchain_openai import ChatOpenAI
from langchain_core.runnables import RunnablePassthrough

# RAG
template = """다음 맥락을 바탕으로 질문에 답변하세요:
{context}

질문: {question}
"""

prompt = ChatPromptTemplate.from_template(template)

llm = ChatOpenAI(model=model_name, temperature=0)

final_rag_chain = (
    {
        "context": retrievel_chain,
        "question": RunnablePassthrough()
    }
    | prompt
    | llm
    | StrOutputParser()
)



In [37]:
final_rag_chain.invoke(question)

'제공된 뉴스 기사들에서는 주로 주식시장 동향, 방위산업 관련주, 건설사 수주 경쟁, 주식거래 앱 불만, 재테크 및 절세 관련 내용이 다뤄지고 있습니다. 다만, 집값의 향방에 대해 직접적으로 언급한 내용은 없습니다.\n\n다만, 건설사들의 재개발 사업 수주 경쟁(예: 성수1지구 시공권 경쟁)과 관련된 기사가 있는데, 이는 특정 지역 재개발 사업이 활발히 진행되고 있음을 시사합니다. 하지만 경기 침체 상황에서 건설사들이 정비사업을 선별적으로 수주하는 추세라는 점도 언급되어 있어, 부동산 시장이 전반적으로 활발하지는 않을 수 있음을 암시합니다.\n\n요약하면, 현재 제공된 정보만으로는 집값의 향방을 명확히 예측하기 어렵고, 일부 재개발 사업은 진행 중이나 경기 침체 영향으로 건설사들이 신중한 수주 전략을 취하고 있다는 점을 참고할 수 있습니다.'